In [ ]:
!python -m pip install OpenTTDLab==0.0.72 pandas==2.2.0

In [ ]:
from openttdlab import run_experiments, bananas_ai, bananas_ai_library

def process_result(result):
    def get_company_value(player):
        try:
            return player['old_economy'][0]['company_value']
        except (KeyError, IndexError):
            return 0
    
    return (
        {
            'date': result['date'],
            'seed': result['experiment']['seed'],
            'terrain_type': result['chunks']['PATS']['0']['difficulty.terrain_type'],
            'name': \
                'Admiral AI' if player['name'].startswith('AdmiralAI') else \
                'trAIns AI' if player['name'].startswith('trAIns AI') else \
                'Unknown',
            'company_value': get_company_value(player),
            'money': player['money'],
        }
        for player in result['chunks']['PLYR'].values()
    )

results = run_experiments(
    openttd_version='13.4',
    opengfx_version='7.1',
    experiments=(
        {
            'seed': seed,
            'ais': (
                # To get a specific version of a library from BaNaNaS, we use the full MD5 rather than
                # the actual version number.
                # trAIns 2.1
                bananas_ai('54524149', 'trAIns', md5='c4c069dc797674e545411b59867ad0c2'),
                # AdmiralAI 25
                bananas_ai('41444d4c', 'AdmiralAI', md5='4ccd92fb8f8f01045145be99a28e14a6', ai_params=(
                    ('use_trains', '1'),
                    ('use_busses', '0'),
                    ('use_trucks', '0'),
                    ('use_planes', '0'),
                )),
            ),
            'days': 366 * 15 + 1,
            'openttd_config': f'''
                [difficulty]
                terrain_type={terrain_type}
                [game_creation]
                starting_year=1960
                quantity_sea_lakes=0
                vehicle_breakdowns=0
                max_x=512
                max_y=512
            ''',
        }
        for seed in range(0, 64)
        for terrain_type in [1, 3]
    ),
    result_processor=process_result,
)

In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df.to_csv('02_trains_ai_vs_admiral_ai_results_01_raw.csv', index=False)